# 라이브러리

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl

import xgboost as xgb
import optuna
from optuna.visualization import plot_optimization_history

from sklearn.metrics import mean_squared_error, mean_absolute_error, confusion_matrix, r2_score, mean_absolute_percentage_error

import random
random.seed(777)

# 시간 옵션 세팅 후 data load

In [ ]:
# ;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;
interval = '10초'  # ;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;
random_seed = 777  # ;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;
# ;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;


month_list = [7,8,9]

if interval == '1초':
    dataset = pd.read_csv(r"D:\1. 음향기반 강우관측\Dataset\음향 데이터\20220731_20221128 데이터셋\XGBoost 학습 데이터셋\sec_Final_dataset_20220731_20221128.csv", encoding='cp949')

elif interval == '10초':
    dataset = pd.read_csv(r"D:\1. 음향기반 강우관측\Dataset\음향 데이터\20220731_20221128 데이터셋\XGBoost 학습 데이터셋\tensec_Final_dataset_20220731_20221128.csv", encoding='cp949')

elif interval == '1분':
    dataset = pd.read_csv(r"D:\1. 음향기반 강우관측\Dataset\음향 데이터\20220731_20221128 데이터셋\XGBoost 학습 데이터셋\min_Final_dataset_20220731_20221128.csv", encoding='cp949')

else:
    print('잘못된 시간입력')


dataset.dropna(inplace=True)
dataset.reset_index(inplace=True, drop=True)

if "rainfall_intensity" in dataset.columns:
    dataset.rename(columns = {'rainfall_intensity':'rainfall intensity'},inplace=True)

if "time" in dataset.columns:
    dataset.rename(columns = {'time':'Time'},inplace=True)



rainfall_threshold = 0.5

for k in tqdm(range(len(dataset))):
    
    if dataset.loc[k,'rainfall intensity'] > rainfall_threshold:
        
        dataset.loc[k, 'rain_OX'] = 1
        
    else:
        dataset.loc[k, 'rain_OX'] = 0

dataset.reset_index(inplace=True)
dataset.head(5)

# 강우 유무 이진분류

## train:valid:test = 3:1:6

In [ ]:
X_train = dataset.iloc[::3,2:6].drop(np.arange(0,len(dataset),12))
y_train = dataset.iloc[::3,-1].drop(np.arange(0,len(dataset),12))
train_index = dataset.iloc[::3,0].drop(np.arange(0,len(dataset),12))

X_valid = dataset.iloc[::12,2:6]
y_valid = dataset.iloc[::12,-1]
valid_index = dataset.iloc[::12,0]

X_test = dataset.drop(np.arange(0,len(dataset),3)).iloc[:,2:6]
y_test = dataset.drop(np.arange(0,len(dataset),3)).iloc[:,-1]
test_index = dataset.drop(np.arange(0,len(dataset),3)).iloc[:,0]

In [ ]:
len(X_train)

In [ ]:
len(X_valid)

In [ ]:
len(X_test)

In [ ]:
(len(X_train) + len(X_valid))*2

## Optuna 하이퍼 파라미터 최적화

In [ ]:
# from sklearn.metrics import f1_score
# import numpy as np

# def f1_eval(y_pred, dtrain):
#     y_true = dtrain.get_label()
#     err = 1-f1_score(y_true, np.round(y_pred))
#     return 'f1_err', err

In [ ]:
def objective(trial):
    """Define the objective function"""

    params = {        
        'eta': trial.suggest_float('eta',0.01,0.1), # 튜닝 필요 
        'gamma' : trial.suggest_float('gamma',0,0.5), # 튜닝 필요
        'max_depth' : trial.suggest_int('max_depth',2,10), # 튜닝 필요 
        'subsample': trial.suggest_float('subsample',0.7,1), # 튜닝 필요
        'min_child_weight':trial.suggest_int('min_child_weight',1,3), # 튜닝 필요
        }

    # Fit the model
    optuna_model = xgb.XGBClassifier(booster='gbtree',
                                     verbosity=0,
                                     n_estimators=5000,
                                     early_stopping_rounds=50,
                                     eval_metric='rmse',
                                     seed=random_seed,
                                     **params)
    
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, 'validation_0-rmse')
    
    optuna_model.fit(X_train,y_train, eval_set=[(X_valid, y_valid)], callbacks=[pruning_callback])

    # Make predictions
    preds = optuna_model.predict(X_test)

    # Evaluate predictions
    rmse = np.sqrt(mean_squared_error(y_test, preds))

    return rmse

sampler = optuna.samplers.TPESampler(seed=777)

study = optuna.create_study(sampler = sampler, direction='minimize')

study.optimize(objective, n_trials=1000)

In [ ]:
print('Number of finished trials: {}'.format(len(study.trials)))
print(' ')
print('Best trial:')
trial = study.best_trial

print('  Value: {}'.format(trial.value))
print(' ')
print('  Params: ')

for key, value in trial.params.items():
    print('    {}: {}'.format(key, value))


plot_optimization_history(study)

## 최적 파라미터 학습 진행

In [ ]:
params = trial.params

xg_class = xgb.XGBClassifier(booster='gbtree', 
                           verbosity=0,
                           n_estimators=5000,
                           early_stopping_rounds=50,
                           eval_metric='rmse',
                           seed=random_seed,
                           **params)
    
xg_class.fit(X_train,y_train, eval_set=[(X_valid, y_valid)])

results = xg_class.evals_result()

In [ ]:
preds = xg_class.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, preds))
print(f"RMSE: {rmse}")

xgb.plot_importance(xg_class)

plt.figure(figsize=(7,5))
plt.plot(results['validation_0']['rmse'], label='train loss', c='gray')
#plt.plot(results['validation_1']['rmse'], label='validation loss', c='red')
plt.legend()
plt.xlabel('n_estimators', size=20, labelpad=15)
plt.ylabel('RMSE', size=20, labelpad=15)
#plt.savefig(r'D:\1. 음향기반 강우관측\SCIE 국외 논문\result_fig/{}_강우 유무 분류 로스 감소.png'.format(interval), bbox_inches='tight', dpi = 600)
plt.plot()

In [ ]:
font = {'family' : 'Arial',
    'weight' : 'medium',
    'size'   : 20,
    'style'  : 'normal'}

mpl.rc('font', **font)

cm = confusion_matrix(y_test, preds)

ax= plt.subplot()
init_cmap = plt.cm.get_cmap("bone")
sns.heatmap(cm, annot=True, fmt='g', ax=ax, cmap=init_cmap.reversed());  #annot=True to annotate cells, ftm='g' to disable scientific notation

# labels, title and ticks
ax.set_xlabel('Predicted classification', size=20, labelpad=15, fontdict={'family' : 'Arial',
    'weight' : 'bold',
    'size'   : 20,
    'style'  : 'normal'})

ax.set_ylabel('True classification', size=20, labelpad=15, fontdict={'family' : 'Arial',
    'weight' : 'bold',
    'size'   : 20,
    'style'  : 'normal'})
    
ax.xaxis.set_ticklabels(['No rain', 'rain'])
ax.yaxis.set_ticklabels(['No rain', 'rain'])
#plt.savefig(r'D:\1. 음향기반 강우관측\SCIE 국외 논문\result_fig/{}_confusion matrix.png'.format(interval), bbox_inches='tight', dpi = 600)
plt.show()

# 강우강도 산정

## New dataset 생성

In [ ]:
X_test['rainfall intensity'] = dataset.drop(np.arange(0,len(dataset),3)).iloc[:,-2]
X_test['preds']=preds
X_test['index']=test_index
new_dataset = X_test[X_test['preds']==1]

new_dataset = new_dataset[['index', 'max_frequency', 'weighted mean_dB', 'mean_dB', 'standard deviation_dB', 'rainfall intensity', 'preds']]

new_dataset.reset_index(inplace=True, drop=True)

new_dataset.head(5)

## train:valid:test = 4:1:5

In [ ]:
new_X_train = new_dataset.drop(np.arange(1,len(new_dataset),2)).drop(np.arange(0,len(new_dataset),10)).iloc[:,1:5]
new_y_train = new_dataset.drop(np.arange(1,len(new_dataset),2)).drop(np.arange(0,len(new_dataset),10)).iloc[:,-2]
new_train_index = new_dataset.drop(np.arange(1,len(new_dataset),2)).drop(np.arange(0,len(new_dataset),10)).iloc[:,0]

new_X_valid = new_dataset.iloc[::10,1:5]
new_y_valid = new_dataset.iloc[::10,-2]
new_valid_index = new_dataset.iloc[::10,0]

new_X_test = new_dataset.iloc[1::2,1:5]
new_y_test = new_dataset.iloc[1::2,-2]
new_test_index = new_dataset.iloc[1::2,0]

In [ ]:
print(f'{len(new_X_train)}, {len(new_X_valid)}, {len(new_X_test)}')

## Optuna 하이퍼파라미터 튜닝

In [ ]:
def objective(trial):
    """Define the objective function"""

    params = {        
        'eta': trial.suggest_float('eta',0.01,0.1), # 튜닝 필요 
        'gamma' : trial.suggest_float('gamma',0,0.5), # 튜닝 필요
        'max_depth' : trial.suggest_int('max_depth',2,10), # 튜닝 필요 
        'subsample': trial.suggest_float('subsample',0.7,1), # 튜닝 필요
        'min_child_weight':trial.suggest_int('min_child_weight',1,3), # 튜닝 필요
        }

    # Fit the model
    optuna_model = xgb.XGBRegressor(booster='gbtree', 
                                    verbosity=0,
                                    n_estimators=5000,
                                    early_stopping_rounds=50,
                                    eval_metric='rmse',
                                    seed=random_seed,
                                    **params)
    
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, 'validation_0-rmse')
    
    optuna_model.fit(new_X_train,new_y_train, eval_set=[(new_X_valid, new_y_valid)], callbacks=[pruning_callback])

    # Make predictions
    new_preds = optuna_model.predict(new_X_test)

    # Evaluate predictions
    new_rmse = np.sqrt(mean_squared_error(new_y_test, new_preds))
    return new_rmse

sampler = optuna.samplers.TPESampler(seed=777)

new_study = optuna.create_study(sampler = sampler, direction='minimize')

new_study.optimize(objective, n_trials=1000)

In [ ]:
print('Number of finished trials: {}'.format(len(new_study.trials)))
print(' ')
print('Best trial:')
new_trial = new_study.best_trial

print('  Value: {}'.format(new_trial.value))
print(' ')
print('  Params: ')

for key, value in new_trial.params.items():
    print('    {}: {}'.format(key, value))

plot_optimization_history(new_study)

## 최적 파라미터 학습 진행

In [ ]:
new_params = new_trial.params

xg_reg = xgb.XGBRegressor(booster='gbtree', 
                          verbosity=0,
                          n_estimators=5000,
                          early_stopping_rounds=50,
                          eval_metric='rmse',
                          seed=random_seed,
                          **new_params)
    
xg_reg.fit(new_X_train,new_y_train, eval_set=[(new_X_valid, new_y_valid)])

results = xg_reg.evals_result()

In [ ]:
new_preds = xg_reg.predict(new_X_test)

new_rmse = np.sqrt(mean_squared_error(new_y_test, new_preds))
print(f"RMSE: {new_rmse}")

xgb.plot_importance(xg_reg)

plt.figure(figsize=(7,5))
plt.plot(results['validation_0']['rmse'], label='train loss', c='gray')
#plt.plot(results['validation_1']['rmse'], label='validation loss', c='red')
plt.legend()
plt.xlabel('n_estimators', size=20, labelpad=15)
plt.ylabel('RMSE', size=20, labelpad=15)
#plt.savefig(r'D:\1. 음향기반 강우관측\SCIE 국외 논문\result_fig/{}_강우강도 산정 로스 감소.png'.format(interval), bbox_inches='tight', dpi = 600)
plt.show()

In [ ]:
plt.figure(figsize=(5,5))
plt.scatter(new_y_test, new_preds, s=1)
plt.xlim(0,50)
plt.ylim(0,50)
plt.xlabel('PARSIVEL', size=20, labelpad=10)
plt.ylabel('SOUND', size=20, labelpad=10)
#plt.savefig(r'D:\1. 음향기반 강우관측\SCIE 국외 논문\result_fig/{}_강우강도 산정 스케터.png'.format(interval), bbox_inches='tight', dpi = 600)
plt.show()

## 무강우 + 산정 강우강도

In [ ]:
aa = pd.DataFrame({'index':new_test_index, 'preds':new_preds})

bb = pd.DataFrame({'index':test_index, 'preds':preds})
bb = bb[bb['preds']==0]

final = pd.concat([aa,bb]).sort_values('index')
final.reset_index(inplace=True, drop=True)

## 시계열 plotting

In [ ]:
## 시간 리스트 생성

pred = final['preds']
ref =  dataset.loc[final['index'].tolist(), 'rainfall intensity'].values

time = []
a = 0

for k in range(len(dataset)):
    if dataset.loc[k,'rainfall intensity'] == ref[a]:
        time.append(dataset.loc[k,'Time'])
        a += 1
        if a == len(ref):
            break

if len(final) != len(time):
    print('********************Error*****************************')

final_df = pd.DataFrame({'Time':np.array(time), 'preds':pred, 'rainfall intensity':ref})

final_df.head(5)

In [ ]:
font = {'family' : 'Arial',
    'weight' : 'medium',
    'size'   : 20,
    'style'  : 'normal'}

mpl.rc('font', **font)

plt.figure(figsize=(20,5))
plt.plot(final_df['Time'],final_df['rainfall intensity'], label='PARSIVEL', c='k',alpha = 0.2)
plt.fill_between(range(len(final_df['rainfall intensity'])), final_df['rainfall intensity'][:len(final_df['rainfall intensity'])], color='k',alpha = 0.3)

plt.fill_between([x for x in final_df['Time'] if x.startswith("2022-11")][0:2], 60, color='black')

plt.plot(final_df['Time'], final_df['preds'], label='This study',linewidth='0.8', c='r')

plt.xticks([x for x in final_df['Time'] if x.endswith('00:00:00') or x.endswith('12:00:00')], size=13)
plt.yticks(size=13)

# plt.fill_between(range(8000,14000), 60,alpha = 0.4, color='#00425A')

# plt.fill_between(range(365,466), 60,alpha = 0.4, color='#1F8A70')

# plt.fill_between(range(780,881), 60,alpha = 0.4, color='#FC7300')


plt.xlim(0,len(final_df))
plt.ylim(0,60)
plt.xlabel('Time', size=25,labelpad=15, fontdict={'family' : 'Arial',
    'weight' : 'bold',
    'size'   : 20,
    'style'  : 'normal'})

plt.ylabel('rainfall intensity (mm/h)', size=25,labelpad=15, fontdict={'family' : 'Arial',
    'weight' : 'bold',
    'size'   : 20,
    'style'  : 'normal'})

plt.legend(fontsize=20, prop={**font})
#plt.savefig(r'D:\1. 음향기반 강우관측\SCIE 국외 논문\result_fig/{}_최종 강우 강도 산정 결과.png'.format(interval), bbox_inches='tight', dpi = 600)
plt.show()

# 정량적 평가지표

In [ ]:
rmse = np.sqrt(mean_squared_error(final_df['rainfall intensity'], final_df['preds']))
print(f"RMSE: {rmse}")

print(f"R squared: {r2_score(final_df['rainfall intensity'], final_df['preds'])}")

mae = mean_absolute_error(final_df['rainfall intensity'], final_df['preds'])
print(f"MAE: {mae}")


np.mean(final_df['rainfall intensity'] - final_df['preds'])

In [ ]:
final_df[final_df['Time'].str.contains('2022-11')]

In [ ]:
font = {'family' : 'Arial',
    'weight' : 'medium',
    'size'   : 20,
    'style'  : 'normal'}

mpl.rc('font', **font)

plt.figure(figsize=(20,8))
plt.plot(final_df['Time'],final_df['rainfall intensity'], label='PARSIVEL', c='k',alpha = 0.2)
plt.fill_between(range(len(final_df['rainfall intensity'])), final_df['rainfall intensity'][:len(final_df['rainfall intensity'])], color='k',alpha = 0.3)

plt.fill_between([x for x in final_df['Time'] if x.startswith("2022-11")][0:2], 60, color='black')


plt.plot(final_df['Time'], final_df['preds'], label='This study', c='r',linewidth='0.8')

#plt.xticks([x for x in final_df['Time'] if x.endswith('00:00:00') or x.endswith('12:00:00') or x.endswith('06:00:00') or x.endswith('18:00:00')], size=22)
plt.xticks([x for x in final_df.loc[np.arange(4282+500,8156,1000), 'Time']], size=23)
plt.tick_params(axis='x', length=10, width=2)

plt.yticks(size=20)

plt.fill_between(range(4790,5290), 60,alpha = 0.6, color='#8F9EAB')

plt.fill_between(range(2240,2740), 60,alpha = 0.6, color='#8F9EAB')

plt.fill_between(range(800,1300), 60,alpha = 0.6, color='#8F9EAB')


plt.xlim(final_df.iloc[4282,0], final_df.iloc[8156,0])
plt.ylim(0,60)
plt.xlabel('Time', size=30, labelpad=15, fontdict={'family' : 'Arial',
    'weight' : 'bold',
    'size'   : 20,
    'style'  : 'normal'})
plt.ylabel('Rainfall Intensity (mm/h)', size=30, labelpad=15, fontdict={'family' : 'Arial',
    'weight' : 'bold',
    'size'   : 20,
    'style'  : 'normal'})

plt.legend(fontsize=30)
plt.savefig(r'D:\1. 음향기반 강우관측\SCIE 국외 논문\result_fig/최종 강우 강도 산정 결과(구분 표시)_non-monsoon.png', bbox_inches='tight', dpi = 600)
plt.show()

In [ ]:
ref = final_df['rainfall intensity']
pred = final_df['preds']

str_ind = 4790
end_ind = 5290

# str_ind = 2240
# end_ind = 2740

# str_ind = 800
# end_ind = 1300

font = {'family' : 'Arial',
    'weight' : 'medium',
    'size'   : 20,
    'style'  : 'normal'}

mpl.rc('font', **font)

plt.figure(figsize=(15,8))

plt.plot(final_df.loc[str_ind:end_ind, 'Time'], ref[str_ind:end_ind+1].cumsum()/360, label='PARSIVEL', linewidth = 1.5, color='k')  # 나누기 60??

plt.plot(final_df.loc[str_ind:end_ind, 'Time'], pred[str_ind:end_ind+1].cumsum()/360, label='This study', linewidth = 1.5, color='r')

plt.xticks([x for x in final_df.loc[np.arange(str_ind+95,end_ind,150), 'Time']], size=23)
plt.yticks(size=20)

plt.ylim(0)

plt.xlim(0,end_ind - str_ind)
plt.ylim(0,11)

plt.ylabel('Precipitation(mm)', size=30, labelpad=15, fontdict={'family' : 'Arial',
    'weight' : 'bold',
    'size'   : 20,
    'style'  : 'normal'})
plt.xlabel('Time', size=30, labelpad=15, fontdict={'family' : 'Arial',
    'weight' : 'bold',
    'size'   : 20,
    'style'  : 'normal'})

plt.grid(linestyle='--')

plt.legend(loc='upper left', fontsize=30, framealpha=1)

plt.savefig(r'D:\1. 음향기반 강우관측\SCIE 국외 논문\result_fig/{1}-{2}_누적 강우강도.png'.format(interval, str_ind, end_ind), bbox_inches='tight', dpi = 600)

plt.show()